matrix = ml.get_matrix()# Autoencoder
- Auto = self
- encode = convert into a different form
- Autoencoder = a system that teaches itself how to encode information
- Number outputs correspond to the entry from the model

# Structure of the layers
- Encode: input > layers > bottleneck or latent code (central node)

#  Goal of autoencoder:
- Get the output to match the input closer possible
- data compression of dimension reduction
- data cleaning (denoising, despeckling, occlusion)  
- feature extraction
- anomaly / fraud detection
- pretraining deep or complex models   


In [60]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as weight_init
import torch.utils.data
from torch._export.db.examples.type_reflection_method import A
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from scipy.sparse import coo_matrix
import torch.nn.parallel
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [36]:
from preprocess.preprocess import MovieLens

ml = MovieLens()

In [49]:
# Custom dataset class for loading sparse user-item interaction matrix
class SparseMatrixDataset(Dataset):
    def __init__(self, matrix):
        self.matrix = matrix  # Convert matrix to COO format for efficient indexing

    def __len__(self):
        return len(self.matrix)

    def __getitem__(self, idx):
        return self.matrix[idx]

In [50]:
# train_loader = ml.dataset_encoder(batch_size=32)
n_users, n_items = ml.num_user_item()

In [51]:
class AutoEncoder(nn.Module):
    def __init__(self, n_enc, n_bottle):
        super(AutoEncoder, self).__init__()

        #input layer
        self.input = nn.Linear(n_items, n_enc)

        ### encoder layer
        self.encoding = nn.Linear(n_enc, n_bottle)

        ### bottleneck layer
        self.bottleneck = nn.Linear(n_bottle, n_enc)

        ### decoder layer
        self.decoding = nn.Linear(n_enc, n_items)

    # forward pass
    def forward(self, x):
        x = F.relu(self.input(x))
        x = F.relu(self.encoding(x))
        x = F.relu(self.bottleneck(x))
        y = torch.sigmoid(self.decoding(x))
        return y

In [52]:
# Set hyperparameters
n_enc = 256
n_bottle = 128
batch_size = 32
learning_rate = 0.001
num_epochs = 10

model = AutoEncoder(n_enc, n_bottle)
# Define loss function and optimizer
loss_fun = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [53]:
# Load and preprocess your sparse user-item interaction matrix
# Example:
sparse_matrix = ml.get_matrix().values
matrix_shape = sparse_matrix.shape
train_dataset = SparseMatrixDataset(coo_matrix(sparse_matrix))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [70]:
# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        print(data[2].shape)
        inputs = data[0].to(device, dtype=torch.float)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Calculate the loss
        loss = loss_fun(outputs, inputs)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

torch.Size([32])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x32 and 9724x256)